# 10. Вложенные запросы

# 10.1 Простые вложенные запросы

Выведите id и названия категорий, в которых в наличии есть товары.
Данные отсортируйте по имени категории.

Используйте вложенные запросы.

**Решение:**

```MySQL
SELECT *  FROM categories
WHERE categories.id IN (
    SELECT products.category FROM products WHERE products.count > 0
    )
ORDER BY categories.name ASC;
```

В таблице orders хранятся заказы пользователей, а в users непосредственно сами покупатели. Получите информацию о покупателях, которые сделали самый дорогой заказ.

Заказов с максимальной стоимостью может быть несколько, а значит и покупателей может быть несколько.

Учитывайте только завершенные заказы.
Данные отсортируйте по id покупателей.

**Решение:**

```MySQL
SELECT *  FROM users
WHERE id IN (
    SELECT user_id FROM orders WHERE status='completed' AND amount =
    (SELECT max(amount) FROM orders WHERE status='completed')
    )
ORDER BY id;
```

Получите из таблицы products джинсы, стоимость которых больше средней цены за джинсы.

Выведите id, название и цену.

Данные отсортируйте по цене, а затем по id.

**Решение:**

```MySQL
SELECT products.id, products.name, products.price 
FROM products 
 WHERE category = 1 AND price >  
 (SELECT AVG(products.price) FROM products
  WHERE products.category = 1)
ORDER BY products.price;
```

# 10.2 IN, ANY, ALL

В таблице products хранятся товары, а в categories — категории, к которым они относятся.

Поле category_id является внешним ключом, который указывает на первичный ключ категории.

Получите список овощей, цена которых больше чем ЛЮБАЯ из цен фруктов (дороже хотя бы одного из фруктов).
Выведите название и цену продукта, данные отсортируйте по названию.

**Решение:**

```MySQL
SELECT name, price
FROM products
WHERE price > ANY (
    SELECT price FROM products WHERE category_id = 3)
AND category_id = 9
ORDER BY name;
```

В таблице products хранятся товары, а в categories — категории, к которым они относятся.

Поле category_id является внешним ключом, который указывает на первичный ключ категории.

Магазину нужно заказать только те фрукты, количество которых меньше самого маленького количества овощей. Получите список фруктов, которые необходимо заказать.
Выведите название и количества продукта, данные отсортируйте по названию.

**Решение:**

```MySQL
SELECT name, count FROM products
WHERE count < ALL(
    SELECT count FROM products WHERE category_id = 9)
AND category_id = 3
ORDER BY name;
```

# 10.3 Ключевое слово EXISTS

Выведите id и названия категорий, в которых есть товары.
Данные отсортируйте по имени категории.

Используйте конструкцию EXISTS.

**Решение:**

```MySQL
SELECT*FROM categories
WHERE EXISTS (
    SELECT * FROM products
    WHERE categories.id=products.category AND products.count>0)
ORDER BY name;

```

Таблицы users и roles связаны отношением многие ко многим через таблицу users_roles.

Получите список всех сотрудников, которые не выполняют ни одной роли.
Выведите id, имена и фамилии таких сотрудников. Данные отсортируйте по id.
Используйте вложенные запросы.

**Решение:**

```MySQL
SELECT id, first_name, last_name
FROM users
WHERE NOT EXISTS (
    SELECT id FROM users_roles WHERE users.id = users_roles.user_id
    )
ORDER BY id;

```

База данных музыкального сайта состоит из 4 таблиц: genres, artists, albums и songs.
Получите все песни за 2008 год.
Выведите название песни (name) и номер альбома (album_id).
Данные отсортируйте по названию композиции.

**Решение:**

```MySQL
SELECT name, album_id
FROM songs
WHERE EXISTS (
    SELECT * FROM albums WHERE year = 2008 AND songs.album_id = id
)
ORDER BY name;

```

# 10.4 Запросы, возвращающие несколько столбцов

В таблице products хранятся товары с текущими ценами, а в old_prices — старые цены на эти же товары. Если товар добавили недавно, то в old_prices данных о нем нет.

Получите из таблицы products id, название и текущую стоимость товаров, цены которых были изменены. Данные отсортируйте по id.

**Решение:**

```MySQL
SELECT products.id, products.name, products.price 
FROM products
JOIN old_prices ON products.id = old_prices.product_id
WHERE (products.id, products.price) NOT IN (
    SELECT product_id, price FROM old_prices); 

```

В таблице people содержится список всех людей, а в suspects — подозреваемых.
Получите из people всех подозреваемых основываясь на данных из таблицы suspects.

Данные должны быть отсортированы по id.

**Решение:**

```MySQL
SELECT * FROM people
WHERE (first_name, last_name, age) IN (
    SELECT * FROM suspects 
    );

```

В таблицах first_names и last_names хранятся все разрешенные в системе имена и фамилии, а в таблице people — все зарегистрированные пользователи.

Получите из people всех людей, у которых неверные имена или фамилии.
Выведите id, имя и фамилию. Данные отсортируйте по фамилии.

**Решение:**

```MySQL
SELECT id, first_name, last_name
FROM people
WHERE (first_name, last_name) NOT IN (
    SELECT first_name, last_name FROM first_names, last_names
    )
ORDER BY last_name;

```

# 10.5 Подзапросы в конструкции FROM

На главной странице сайта выводится ТОП5 компьютерных игр. Чтобы вывод был разнообразным:

1. Берут по две самых популярных игры из 5 категорий.
2. Из полученной десятки оставляют пять лучших игр.
3. Итоговую пятерку сортируют по рейтингу в прямом порядке.

Строки с одинаковым рейтингом отсортируйте по id.

Получите из таблицы games данные для вывода на главную страницу сайта для категорий: 1 - Action, 2 - RPG, 3 - Adventure, 4 - Strategy и 5 - Shooter. Выведите поля id, name, rating и genre, где genre — название категории.

**Решение:**

```MySQL
SELECT id, name, rating, genre
FROM (
     (SELECT *,'Action' AS genre FROM games WHERE category_id = 1 ORDER BY rating DESC LIMIT 2)
UNION
     (SELECT *,'RPG' AS genre  FROM games WHERE category_id = 2 ORDER BY rating DESC LIMIT 2)
UNION
     (SELECT *,'Adventure' AS genre  FROM games WHERE category_id = 3 ORDER BY rating DESC LIMIT 2)
UNION
     (SELECT *,'Strategy' AS genre  FROM games WHERE category_id = 4 ORDER BY rating DESC LIMIT 2)
UNION
     (SELECT *,'Shooter' AS genre  FROM games WHERE category_id = 5 ORDER BY rating DESC LIMIT 2)
ORDER BY rating DESC LIMIT 5
     ) AS bg
ORDER BY rating, id;
```

Для хранения транзакций в базе данных используется 3 таблицы: bank_transactions, cashbox_transactions и paypal_transactions.

Выведите дату (колонка date), сумму и тип последних трех транзакций.
Итоговые данные отсортируйте по дате в хронологическом порядке.
В качестве типа используйте bank, cash и paypal в зависимости от таблицы.
Тип выведите в поле payment_type.

**Решение:**

```MySQL
SELECT * FROM (
    SELECT date, amount, payment_type FROM (
        (SELECT *, 'bank' as payment_type FROM bank_transactions)
        UNION
        (SELECT *, 'cash' FROM cashbox_transactions)
        UNION
        (SELECT *, 'paypal' FROM paypal_transactions)
    ) as all_transactions_DESC
    ORDER BY date DESC
    LIMIT 3) as all_transactions
ORDER BY date;
```

В таблицах first_names и last_names хранятся все доступные в системе имена и фамилии. Получите все возможные комбинации фамилий и имен. Фамилии мужского пола могут сочетаться только с мужскими именами, тоже касается и женских фамилий и имен.

В итоговой таблице выведите фамилии, имена и пол. Данные отсортируйте сперва по полу, а затем по фамилии и имени.

**Решение:**

```MySQL
SELECT l.last_name, f.first_name, f.sex
FROM first_names AS f
JOIN last_names AS l ON f.sex = l.sex
ORDER BY sex,last_name,first_name;
```

# 10.6 Подзапросы в конструкции INSERT

В базе данных есть две таблицы для хранения объявлений: advs и closed_advs, в первой хранятся все объявления, а во второй только неактивные.

Раз в день объявления из таблицы advs копируются в closed_advs — некоторые объявления уже скопированы.
Скопируйте оставшиеся объявления в таблицу closed_advs.

**Решение:**

```MySQL
INSERT IGNORE INTO closed_advs (
    SELECT id, user_id, category_id, text, date FROM advs
    WHERE closed = 1);
```

База данных автосалона содержит 4 таблицы: marks, models, cars и cached_cars.
В marks и models хранятся данные о марках и моделях автомобилей, а в cars сами автомобили.

Для ускорения доступа к данным в базу добавили промежуточную таблицу cached_cars, которая хранит уже готовые строки в виде «марка модель, цвет», а также текущую цену автомобиля.

Заполните таблицу cached_cars свежими данными. Обратите внимание, что некоторые данные в таблице cached_cars уже устарели и их нужно обновить.

**Решение:**

```MySQL
REPLACE INTO cached_cars (SELECT
    cars.id as id,
    CONCAT(marks.name," ",models.name,', ',cars.color) AS car,
    cars.price as price
FROM 
    cars
JOIN models ON models.id = cars.model_id
JOIN marks ON marks.id = models.mark_id);
```

База данных автосалона содержит 4 таблицы: marks, models, cars и cached_cars.
В marks и models хранятся данные о марках и моделях автомобилей, а в cars сами автомобили.

Для ускорения доступа к данным в базу добавили промежуточную таблицу cached_cars, которая хранит уже готовые строки в виде «марка модель, цвет», а также текущую цену автомобиля.

Заполните таблицу cached_cars свежими данными. Обратите внимание, что некоторые данные в таблице cached_cars уже устарели и их нужно обновить.

**Решение:**

```MySQL
REPLACE INTO cached_cars (SELECT
    cars.id as id,
    CONCAT(marks.name," ",models.name,', ',cars.color) AS car,
    cars.price as price
FROM 
    cars
JOIN models ON models.id = cars.model_id
JOIN marks ON marks.id = models.mark_id);
```

# THE END